# USB Webcam

This notebook shows how to use a USB web camera attached to the board. An image is captured using [fswebcam](http://manpages.ubuntu.com/manpages/wily/man1/fswebcam.1.html). The image can then be manipulated using the Python Image Library (Pillow).

The webcam used is the Logitech USB HD Webcam C270 and the driver for this webcam has already been installed on the board.

#### References
http://pillow.readthedocs.org/en/3.1.x/handbook/tutorial.html<br>
http://manpages.ubuntu.com/manpages/lucid/man1/fswebcam.1.html <br>
http://www.logitech.com/en-us/product/hd-webcam-c270

In [3]:
#Downgrade to lower version for package support

!pip install "numpy<2"

In [ ]:
#Install yolo if already havent
!pip install ultralytics
!pip install opencv-python

In [2]:
from ultralytics import YOLO
import os
import cv2

model = YOLO('LeafAI.pt')

In [4]:
#THIS CODE WAS PARTIALLY WRITTEN WITH THE HELP OF GEMINI AI, THE MAIN REASON DUE TO AN ERROR WITH IMAGE PROCESSING.

import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import io
import time
import IPython
from IPython.display import display, clear_output
import ipywidgets as widgets

frame = 0
class_name = 0
confidence = 0

Image_widget = widgets.Image(
    format = 'jpeg',
    width = 300,
    height = 200
)

display(Image_widget)

def get_frame(cam):
    success, frame = cam.read()
    if not success:
        return None
    return frame

def array_to_image_bytes(frame_rgb):#Fixes the buf bug
    pil_image = Image.fromarray(frame_rgb)
    f = io.BytesIO()
    pil_image.save(f, format='jpeg')
    return f.getvalue()
    
    
cam = cv2.VideoCapture(0)
d = IPython.display.display("",display_id=1)

try:
    while True:
        frame = get_frame(cam)
        if frame is None:
            print("frame dropped")
            break
            
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        img_bytes = array_to_image_bytes(frame_rgb)
        
        Image_widget.value = img_bytes
except KeyboardInterrupt:
    results = model(frame)
    ConfName = ""
    ConfVal = 0
    for r in results:
        boxes = r.boxes

        im_bgr = r.plot()

        im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)

        plt.figure(figsize=(12,8))
        plt.axis('off')
        plt.imshow(im_rgb)

        plt.show()

        for box in boxes:
            class_id = box.cls.item()
            class_name = model.names[class_id]


            confidence = box.conf.item()

            if confidence > ConfVal:
                ConfVal = confidence
                ConfName = class_name

    print(f"image is: {ConfName} with confidence: {ConfVal:.2f}")
finally:
    cam.release()





KeyboardInterrupt

